In [2]:
import pandas as pd
import numpy as np
from time import time
from collections import Counter
import collections

# natural language processing: n-gram ranking
import re
import unicodedata
import nltk
import spacy
import difflib

In [3]:
df_train = pd.read_csv('data/processed_train.csv')
df_train.head()

,raw_address,POI/street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung
2,"toko dita , kertosono",toko dita/
3,jl. orde baru,/jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede


In [4]:
def getInterest(_list, arr_index):
    return _list[arr_index]

df_train['POI'] = df_train['POI/street'].str.split('/').apply(getInterest, args=(0,))
df_train['street'] = df_train['POI/street'].str.split('/').apply(getInterest, args=(1,))
df_train.head(20)

,raw_address,POI/street,POI,street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
2,"toko dita , kertosono",toko dita/,toko dita,
3,jl. orde baru,/jl. orde baru,,jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede,toko bb kids,raya samb gede
5,"kem mel raya , no 4 bojong rawalumbu rt 1 36 r...",/kem mel raya,,kem mel raya
6,tela keuramat kuta alam,/tela,,tela
7,gg. i wates magersari,/gg. i,,gg. i
8,bunga ncole ix 2,/bunga ncole ix,,bunga ncole ix
9,"cikahuripan sd neg boj 02 klap boj , no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj


In [5]:
def getListOfWords(sentence):
    sentence = sentence.replace('\n', ' ').replace('\t', ' ')
    sentence = sentence.replace('/', ' ').replace('.', ' ').replace(',', ' ')
    words = sentence.split(' ')
    while '' in words:
        words.remove('')
        
    return [word for word in words if len(words) > 0]

In [6]:
def getDifference(row):
    raw = getListOfWords(row['raw_address'])
    poi_street = getListOfWords(row['POI/street'])
    temp = set(raw)
    output = [x for x in poi_street if x not in temp]
    if len(output) > 0:
        return output
    else:
        return np.nan

In [7]:
start = time()

df_train['delta_words'] = df_train.apply(getDifference, axis=1)

print("Executed in {} seconds.".format(time() - start))

Executed in 3.891024351119995 seconds.


In [8]:
df_need_repair = df_train.dropna()
df_need_repair

,raw_address,POI/street,POI,street,delta_words
9,"cikahuripan sd neg boj 02 klap boj , no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"[negeri, bojong]"
10,"yaya atohar ,",yayasan atohariyah/,yayasan atohariyah,,"[yayasan, atohariyah]"
18,"toko bang ajs ,",toko bangunan ajs/,toko bangunan ajs,,[bangunan]
36,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,"[markaz, tabligh]"
38,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,[negeri]
...,...,...,...,...,...
265866,moha toha no 167,/mohammad toha,,mohammad toha,[mohammad]
265867,"islamic training cen pare , brawi , pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,[center]
265873,"la banda minima , cile raya , pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,[minimarket]
265875,"ginzi cafe , siliw ,",ginzi cafe/siliwangi,ginzi cafe,siliwangi,[siliwangi]


# N-gram analysis

## find the pair

In [9]:
def getDistance(str1, str2):
    # return nltk.edit_distance(str1, str2)
    seq = difflib.SequenceMatcher(None,str1,str2)
    similarity = seq.ratio()
    distance = 1/similarity if similarity>0 else 99999
    return distance
    
getDistance('rakyat', 'makan')

1.8333333333333335

In [10]:
def getPair(row):
    raw = row['raw_address']
    repaired = row['delta_words']
    _dict1 = {}
    _dict2 = {}
    for repaired_word in repaired:
        _temp = {}
        _dictBackup = {}
        for raw_word in getListOfWords(raw):
            _dictBackup[raw_word] = getDistance(repaired_word, raw_word)
            if repaired_word[0] == raw_word[0]:
                _temp[raw_word] = getDistance(repaired_word, raw_word)
        if len(_temp) == 0:
            _temp = _dictBackup
        _dict1[repaired_word] = _temp
    
    for key_repaired, value_raw_dict in _dict1.items():
        # get minimum distance word
        sorted_dict2 = sorted(value_raw_dict.items(), key=lambda kv: kv[1])
        _dict2[key_repaired] = sorted_dict2[0][0]
    # inverse dict from raw to repaired
    inv_map = {v: k for k, v in _dict2.items()}
    return inv_map

In [11]:
df_need_repair['Pair'] = df_need_repair.apply(getPair, axis=1)
df_need_repair

<ipython-input-11-36cfc8fdbb57>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_repair['Pair'] = df_need_repair.apply(getPair, axis=1)


,raw_address,POI/street,POI,street,delta_words,Pair
9,"cikahuripan sd neg boj 02 klap boj , no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"[negeri, bojong]","{'neg': 'negeri', 'boj': 'bojong'}"
10,"yaya atohar ,",yayasan atohariyah/,yayasan atohariyah,,"[yayasan, atohariyah]","{'yaya': 'yayasan', 'atohar': 'atohariyah'}"
18,"toko bang ajs ,",toko bangunan ajs/,toko bangunan ajs,,[bangunan],{'bang': 'bangunan'}
36,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,"[markaz, tabligh]","{'mar': 'markaz', 'tabl': 'tabligh'}"
38,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,[negeri],{'neg': 'negeri'}
...,...,...,...,...,...,...
265866,moha toha no 167,/mohammad toha,,mohammad toha,[mohammad],{'moha': 'mohammad'}
265867,"islamic training cen pare , brawi , pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,[center],{'cen': 'center'}
265873,"la banda minima , cile raya , pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,[minimarket],{'minima': 'minimarket'}
265875,"ginzi cafe , siliw ,",ginzi cafe/siliwangi,ginzi cafe,siliwangi,[siliwangi],{'siliw': 'siliwangi'}


In [12]:
corpus = []
for index, row in df_need_repair.iterrows():
    for key, value in row['Pair'].items():
        temp = [key,value]
        corpus.append(temp)
corpus = Counter(tuple(item) for item in corpus)
corpus = dict(sorted(corpus.items(), key=lambda item: item[1], reverse=True))

In [13]:
dict_1gram = {}
for key, value in corpus.items():
    if key[0] in dict_1gram:
        dict_1gram[key[0]].append({key[1]: value})
    else:
        dict_1gram[key[0]] = [{key[1]: value}]
dict_1gram

{'war': [{'warung': 1535},
  {'warkop': 217},
  {'wartel': 82},
  {'warteg': 67},
  {'warnet': 56},
  {'warnas': 6},
  {'warunk': 6},
  {'warkom': 5},
  {'wardah': 4},
  {'warong': 2},
  {'waronk': 2},
  {'warudo': 2},
  {'warsih': 2},
  {'warnes': 1},
  {'wasrun': 1},
  {'waries': 1},
  {'warnis': 1},
  {'warsab': 1},
  {'wareeg': 1},
  {'wariah': 1},
  {'warsan': 1},
  {'warsat': 1},
  {'warles': 1},
  {'wareng': 1},
  {'wartek': 1},
  {'warter': 1},
  {'warkan': 1},
  {'warlap': 1},
  {'wartaw': 1},
  {'warsun': 1},
  {'warbon': 1}],
 'neg': [{'negeri': 1057}, {'negara': 103}],
 'mas': [{'masjid': 1055},
  {'master': 26},
  {'masehi': 6},
  {'masnur': 2},
  {"mas'ud": 2},
  {'masagi': 1},
  {'masayu': 1},
  {'masdah': 1},
  {'maskur': 1},
  {'masuya': 1},
  {'massip': 1},
  {'massie': 1},
  {'masoem': 1},
  {'mastun': 1},
  {'muyass': 1},
  {'mastur': 1},
  {'maskan': 1},
  {'mastam': 1},
  {'masken': 1},
  {'masugi': 1},
  {'masril': 1},
  {'maskos': 1},
  {'maslah': 1},
  {'mashur

## Find bigram pair

In [14]:
for ngram in nltk.ngrams(getListOfWords("cikahuripan sd neg boj 02 klap boj, no 5 16877"), 2):
    _str = ' '.join(ngram)
    print ('{} -> sd negeri : {}'.format(_str, getDistance(_str, 'sd negeri')))

cikahuripan sd -> sd negeri : 5.75
sd neg -> sd negeri : 1.25
neg boj -> sd negeri : 2.6666666666666665
boj 02 -> sd negeri : 7.5
02 klap -> sd negeri : 8.0
klap boj -> sd negeri : 8.5
boj no -> sd negeri : 3.75
no 5 -> sd negeri : 6.5
5 16877 -> sd negeri : 8.0


In [15]:
def getDifferenceBigram(row):
    _pair = row['Pair']
    raw_words = getListOfWords(row['raw_address'])
    output = []
    _length = len(raw_words)
    for index, value in enumerate(raw_words):
        if value in _pair:
            # check if first index
            if index != 0:
                output.append(raw_words[index-1] + ' ' + _pair[value])   
            # check if last index
            if index != _length-1:
                output.append(_pair[value] + ' ' + raw_words[index+1])
    return output

In [16]:
start = time()

df_need_repair['bigram_diff'] = df_need_repair.apply(getDifferenceBigram, axis=1)

print("Executed in {} seconds.".format(time() - start))

Executed in 0.8966429233551025 seconds.


<ipython-input-16-13f2a1e05b88>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_repair['bigram_diff'] = df_need_repair.apply(getDifferenceBigram, axis=1)


In [17]:
df_need_repair.tail()

,raw_address,POI/street,POI,street,delta_words,Pair,bigram_diff
265866,moha toha no 167,/mohammad toha,,mohammad toha,[mohammad],{'moha': 'mohammad'},[mohammad toha]
265867,"islamic training cen pare , brawi , pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,[center],{'cen': 'center'},"[training center, center pare]"
265873,"la banda minima , cile raya , pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,[minimarket],{'minima': 'minimarket'},"[banda minimarket, minimarket cile]"
265875,"ginzi cafe , siliw ,",ginzi cafe/siliwangi,ginzi cafe,siliwangi,[siliwangi],{'siliw': 'siliwangi'},[cafe siliwangi]
265879,"ver , durian bajenis",veruna/,veruna,,[veruna],{'ver': 'veruna'},[veruna durian]


In [18]:
def getPairBigram(row):
    raw = row['raw_address']
    repaired = row['bigram_diff']
    _dict1 = {}
    _dict2 = {}
    for repaired_word in repaired:
        _temp = {}
        _dictBackup = {}
        for ngram in nltk.ngrams(getListOfWords(raw),2):
            _str = ' '.join(ngram)
            _dictBackup[_str] = getDistance(repaired_word, _str)
            if repaired_word[0] == _str[0]:
                _temp[_str] = getDistance(repaired_word, _str)
        if len(_temp) == 0:
            _temp = _dictBackup
        _dict1[repaired_word] = _temp
    
    for key_repaired, value_raw_dict in _dict1.items():
        # get minimum distance word
        sorted_dict2 = sorted(value_raw_dict.items(), key=lambda kv: kv[1])
        _dict2[key_repaired] = sorted_dict2[0][0]
    # inverse dict from raw to repaired
    inv_map = {v: k for k, v in _dict2.items()}
    return inv_map

In [19]:
start = time()

df_need_repair['PairBigram'] = df_need_repair.apply(getPairBigram, axis=1)

print("Executed in {} minutes.".format(round((time() - start)/60,3)))
df_need_repair

Executed in 0.343 minutes.


<ipython-input-19-06e35669a883>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_need_repair['PairBigram'] = df_need_repair.apply(getPairBigram, axis=1)


,raw_address,POI/street,POI,street,delta_words,Pair,bigram_diff,PairBigram
9,"cikahuripan sd neg boj 02 klap boj , no 5 16877",sd negeri bojong 02/klap boj,sd negeri bojong 02,klap boj,"[negeri, bojong]","{'neg': 'negeri', 'boj': 'bojong'}","[sd negeri, negeri boj, neg bojong, bojong 02,...","{'sd neg': 'sd negeri', 'neg boj': 'neg bojong..."
10,"yaya atohar ,",yayasan atohariyah/,yayasan atohariyah,,"[yayasan, atohariyah]","{'yaya': 'yayasan', 'atohar': 'atohariyah'}","[yayasan atohar, yaya atohariyah]",{'yaya atohar': 'yaya atohariyah'}
18,"toko bang ajs ,",toko bangunan ajs/,toko bangunan ajs,,[bangunan],{'bang': 'bangunan'},"[toko bangunan, bangunan ajs]","{'toko bang': 'toko bangunan', 'bang ajs': 'ba..."
36,mar tabl metro iringmulyo metro timur,markaz tabligh metro/,markaz tabligh metro,,"[markaz, tabligh]","{'mar': 'markaz', 'tabl': 'tabligh'}","[markaz tabl, mar tabligh, tabligh metro]","{'mar tabl': 'mar tabligh', 'tabl metro': 'tab..."
38,sd neg 12 anggrek,sd negeri 12 anggrek/,sd negeri 12 anggrek,,[negeri],{'neg': 'negeri'},"[sd negeri, negeri 12]","{'sd neg': 'sd negeri', 'neg 12': 'negeri 12'}"
...,...,...,...,...,...,...,...,...
265866,moha toha no 167,/mohammad toha,,mohammad toha,[mohammad],{'moha': 'mohammad'},[mohammad toha],{'moha toha': 'mohammad toha'}
265867,"islamic training cen pare , brawi , pelem pare",islamic training center pare/brawi,islamic training center pare,brawi,[center],{'cen': 'center'},"[training center, center pare]","{'training cen': 'training center', 'cen pare'..."
265873,"la banda minima , cile raya , pesanggrahan",la banda minimarket/cile raya,la banda minimarket,cile raya,[minimarket],{'minima': 'minimarket'},"[banda minimarket, minimarket cile]","{'banda minima': 'banda minimarket', 'minima c..."
265875,"ginzi cafe , siliw ,",ginzi cafe/siliwangi,ginzi cafe,siliwangi,[siliwangi],{'siliw': 'siliwangi'},[cafe siliwangi],{'cafe siliw': 'cafe siliwangi'}


In [20]:
corpus2gram = []
for index, row in df_need_repair.iterrows():
    for key, value in row['PairBigram'].items():
        temp = [key,value]
        corpus2gram.append(temp)
corpus2gram = Counter(tuple(item) for item in corpus2gram)
corpus2gram = dict(sorted(corpus2gram.items(), key=lambda item: item[1], reverse=True))

dict_2gram = {}
for key, value in corpus2gram.items():
    if key[0] in dict_2gram:
        dict_2gram[key[0]].append({key[1]: value})
    else:
        dict_2gram[key[0]] = [{key[1]: value}]
dict_2gram

{'sd neg': [{'sd negeri': 715}],
 'war makan': [{'warung makan': 201}, {'war masakan': 1}],
 'smp neg': [{'smp negeri': 169}],
 'mas al': [{'masjid al': 152}],
 'kan desa': [{'kantor desa': 145}, {'kangin desa': 1}],
 'neg 1': [{'negeri 1': 128}, {'negara 1': 1}],
 'jend sudi': [{'jend sudirman': 124},
  {'jenderal sudi': 74},
  {'jendral sudi': 18},
  {'jend sudiarto': 1},
  {'jend studio': 1}],
 'war nasi': [{'warung nasi': 117}, {'wasrun nasi': 1}, {'warsih nasi': 1}],
 'beng las': [{'bengkel las': 105}, {'benteng las': 1}],
 'toko kelon': [{'toko kelontong': 102}],
 'nasi gor': [{'nasi goreng': 101}],
 'neg 2': [{'negeri 2': 99}],
 'taman pendid': [{'taman pendidikan': 93}],
 'war kopi': [{'warung kopi': 87}, {'warong kopi': 1}, {'warkop kopi': 1}],
 'mas nurul': [{'masjid nurul': 83}],
 'bank rak': [{'bank rakyat': 81}],
 'mas jami': [{'masjid jami': 80}, {'mas jamilega': 1}, {'mas jamiatul': 1}],
 'peri kem': [{'peri kemerd': 69}, {'perintis kem': 13}],
 'bank mand': [{'bank mand

### Find Ambigous Pairs

In [21]:
normalized_thresh = 0.8
ambigous_pairs = []
for key, val in dict_2gram.items():
    above_thresh = 0
    _index = 0
    for element in val:
        for pair_key,count in element.items():
            if _index == 0:
                _max = count
            if count/_max > normalized_thresh:
                above_thresh += 1
        _index += 1
    if above_thresh > 1 and _max > 1:
        ambigous_pairs.append(key)

In [22]:
len(ambigous_pairs)

72

In [23]:
ambigous_pairs[:5]

['bas rah', 'sla riy', 'tent pela', 'pang sudi', 'wij kus']

In [24]:
# tags = ['wij kus', 'brig kata', 'mar raya']
tags = ambigous_pairs
for tag in tags:
    print('{}\t --> {}'.format(tag, dict_2gram[tag]))

bas rah	 --> [{'bas rahmat': 22}, {'basuki rah': 21}, {'bas rahmad': 1}]
sla riy	 --> [{'sla riyadi': 21}, {'slamet riy': 17}]
tent pela	 --> [{'tent pelajar': 19}, {'tentara pela': 16}]
pang sudi	 --> [{'pang sudirman': 17}, {'panglima sudi': 14}]
wij kus	 --> [{'wijaya kus': 15}, {'wij kusuma': 14}]
pang anta	 --> [{'pangeran anta': 9}, {'pang antasari': 8}]
prak dokter	 --> [{'praktek dokter': 9}, {'praktik dokter': 8}]
has no	 --> [{'hasyim no': 9}, {'hasanu no': 9}, {'hassan no': 1}]
unt suro	 --> [{'unt suropati': 7}, {'untung suro': 6}]
raya pan	 --> [{'raya pandeg': 6}, {'raya pangan': 5}, {'raya pantai': 4}, {'raya pangal': 3}, {'raya panumb': 1}, {'raya pandan': 1}]
brig sla	 --> [{'brigjen sla': 6}, {'brig slamet': 5}]
foto raya	 --> [{'fotokopi raya': 6}, {'fotocopy raya': 6}]
copy cen	 --> [{'copy centre': 6}, {'copy center': 5}]
sul iska	 --> [{'sultan iska': 5}, {'sul iskanda': 5}, {'sul iskandar': 1}]
raya kal	 --> [{'raya kalima': 5}, {'raya kaliga': 5}, {'raya kalija'

In [25]:
key = 'wij kus'
val = 'wij kusuma'
for index, row in df_need_repair.iterrows():
    if key in row['PairBigram']:
        if row['PairBigram'][key] == val:
            print('idx: {}'.format(index))
            print('raw: {}'.format(row['raw_address']))
            print('poi/street: {}'.format(row['POI/street']))
            print('repaired words: {}'.format(row['delta_words']))
            print('---> Pair1gram: {}'.format(row['Pair']))
            print('---> Pair2gram: {}'.format(row['PairBigram']))
            print('*'*50)

idx: 27425
raw: rumah dim , wij kus raya , no 63 bogor barat - kota
poi/street: rumah dimsum/wij kusuma raya
repaired words: ['dimsum', 'kusuma']
---> Pair1gram: {'dim': 'dimsum', 'kus': 'kusuma'}
---> Pair2gram: {'rumah dim': 'rumah dimsum', 'dim wij': 'dimsum wij', 'wij kus': 'wij kusuma', 'kus raya': 'kusuma raya'}
**************************************************
idx: 63399
raw: tk wij kus , jati , surodadi
poi/street: tk wijaya kusuma/jati
repaired words: ['wijaya', 'kusuma']
---> Pair1gram: {'wij': 'wijaya', 'kus': 'kusuma'}
---> Pair2gram: {'tk wij': 'tk wijaya', 'wij kus': 'wij kusuma', 'kus jati': 'kusuma jati'}
**************************************************
idx: 82688
raw: malaka sari wij kus i , 1 rt 1 7
poi/street: /wij kusuma i
repaired words: ['kusuma']
---> Pair1gram: {'kus': 'kusuma'}
---> Pair2gram: {'wij kus': 'wij kusuma', 'kus i': 'kusuma i'}
**************************************************
idx: 104840
raw: mega toy's wij kus , jaka sampurna bekasi barat
poi/

# Combine ngrams

In [26]:
tokens = []
for s in df_train["raw_address"] :
    words = s.split(" ")
    for word in words :
        tokens.append(word)
count = Counter(tokens) 

In [27]:
small_dict = {}
for k, v in dict_1gram.items() :
    first_word, first_count = list(v[0].items())[0]
    uncertain = True
    if len(v) > 1 :
        second_word, second_count = list(v[1].items())[0]
        if first_count < second_count * 5 :
            uncertain = False
    else :
        uncertain = False

    if not uncertain :
        if count[k] < 2 * first_count :
            small_dict[k] = first_word

In [28]:
bigrams = dict_2gram
unigrams = dict_1gram

In [29]:
bigrams["neg boj"]

[{'neg bojong': 2}]

In [30]:
unigrams["neg"]

[{'negeri': 1057}, {'negara': 103}]

# Checking

In [37]:
df = pd.read_csv('data/processed_train.csv')
df.head()

,raw_address,POI/street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung
2,"toko dita , kertosono",toko dita/
3,jl. orde baru,/jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede


In [38]:
def repair_bigram(text) :
    words = text.split(" ")
    n = len(words)
    if n == 1 :
        return text
    replaced = [False] * n
    repaired = words.copy()
    i = 0
    while i < n - 1 :
        token = words[i] + " " + words[i+1] 
#         if replaced[i] :
#             token = repaired[i] + " " + words[i+1] 
        if token in bigrams :
            sol = bigrams[token][0]
            k = list(sol.keys())[0]
            w1, w2 = k.split(" ")
            if w1 != words[i] :
                replaced[i] = True
                repaired[i] = w1
            if w2 != words[i+1] :
                replaced[i+1] = True
                repaired[i+1] = w2
        i += 1

    return " ".join(repaired)
        
text = "cikahuripan sd neg boj 02 klap boj , no 5 16877"
repair_bigram(text)

'cikahuripan sd negeri bojong 02 klap bojong , no 5 16877'

In [39]:
bigrams

{'sd neg': [{'sd negeri': 715}],
 'war makan': [{'warung makan': 201}, {'war masakan': 1}],
 'smp neg': [{'smp negeri': 169}],
 'mas al': [{'masjid al': 152}],
 'kan desa': [{'kantor desa': 145}, {'kangin desa': 1}],
 'neg 1': [{'negeri 1': 128}, {'negara 1': 1}],
 'jend sudi': [{'jend sudirman': 124},
  {'jenderal sudi': 74},
  {'jendral sudi': 18},
  {'jend sudiarto': 1},
  {'jend studio': 1}],
 'war nasi': [{'warung nasi': 117}, {'wasrun nasi': 1}, {'warsih nasi': 1}],
 'beng las': [{'bengkel las': 105}, {'benteng las': 1}],
 'toko kelon': [{'toko kelontong': 102}],
 'nasi gor': [{'nasi goreng': 101}],
 'neg 2': [{'negeri 2': 99}],
 'taman pendid': [{'taman pendidikan': 93}],
 'war kopi': [{'warung kopi': 87}, {'warong kopi': 1}, {'warkop kopi': 1}],
 'mas nurul': [{'masjid nurul': 83}],
 'bank rak': [{'bank rakyat': 81}],
 'mas jami': [{'masjid jami': 80}, {'mas jamilega': 1}, {'mas jamiatul': 1}],
 'peri kem': [{'peri kemerd': 69}, {'perintis kem': 13}],
 'bank mand': [{'bank mand

In [40]:
def repair_unigram(text) :
    words = text.split(" ")
    for i in range(len(words)) :
        if words[i] in small_dict :
            words[i] = small_dict[words[i]]
    return " ".join(words)

In [41]:
repair_unigram("negeri")

'negeri'

In [42]:
def repair(text) : 
    text = repair_bigram(text)
    text = repair_unigram(text)
    return text

In [43]:
df["repaired_address"] = df["raw_address"].apply(repair)

In [44]:
df

,raw_address,POI/street,repaired_address
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,jl kapuk timur delta silikon iii lippo cikaran...
1,setu siung 119 rt 5 1 13880 cipayung,/siung,setu siung 119 rt 5 1 13880 cipayung
2,"toko dita , kertosono",toko dita/,"toko dita , kertosono"
3,jl. orde baru,/jl. orde baru,jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede,"raya sambong gede , 299 toko bb kids"
...,...,...,...
265883,karawaci baru kakap raya 156 rt 1 rw 3 karawaci,/kakap raya,karawaci baru kakap raya 156 per 1 rw 3 karawaci
265884,jend ahmad yani 331 kertasari ciamis,/jend ahmad yani,jenderal mas yani 331 kertasari ciamis
265885,"raya cila kko , cilandak timur kel.",/raya cila kko,"raya cilandak kko , cilandak timur kel."
265886,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri/,jalan cipadu jaya taman asri gang bijaksana 3 ...


### Before Repair

In [45]:
check = df["raw_address"] == cdf["raw_address"]
sum(check)/len(check)

NameError: name 'cdf' is not defined

### After Repair

In [ ]:
check = df["repaired_address"] == cdf["raw_address"]
sum(check)/len(check)

In [ ]:
df["raw_address"] = df["repaired_address"]
df[["id","raw_address"]].to_csv("data/cleaned_test.csv", index=False)

In [46]:
cdf = pd.read_csv('data/cleaned_train.csv')
cdf.head()

,raw_address,POI/street
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika
1,setu siung 119 rt 5 1 13880 cipayung,/siung
2,"toko dita , kertosono",toko dita/
3,jl. orde baru,/jl. orde baru
4,"raya samb gede , 299 toko bb kids",toko bb kids/raya samb gede


In [47]:
def f(text, ps) :
    return ps.split("/")[0] in text and ps.split("/")[1] in text

check = cdf.apply(lambda x: f(x.raw_address, x["POI/street"]), axis=1)

In [48]:
len(cdf)

265888

In [49]:
sum(check)

265523

In [50]:
check

0         True
1         True
2         True
3         True
4         True
          ... 
265883    True
265884    True
265885    True
265886    True
265887    True
Length: 265888, dtype: bool